# Breast Cancer Classification

# Import packages

In [126]:
#!pip install scikit-learn opencv-python 

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f694a4ec10, raw_cell="#!pip install scikit-learn opencv-python " store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W2sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a5399190, execution_count=126 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f694a4ec10, raw_cell="#!pip install scikit-learn opencv-python " store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W2sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [127]:
#pip install torch==2.0+cu117 --user -f https://download.pytorch.org/whl/cu117/torch_stable.html

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a53d0550, raw_cell="#pip install torch==2.0+cu117 --user -f https://do.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W3sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a5372d90, execution_count=127 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a53d0550, raw_cell="#pip install torch==2.0+cu117 --user -f https://do.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W3sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [128]:
#!pip install torchvision==0.15.0

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f6a3dd9f10, raw_cell="#!pip install torchvision==0.15.0" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W4sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a53a9f50, execution_count=128 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f6a3dd9f10, raw_cell="#!pip install torchvision==0.15.0" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W4sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [129]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import glob
#import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import random
import cv2
import sys
from torchvision import models,transforms,datasets
import time
import os
import torch.nn.functional as F
from torchvision.datasets.folder import default_loader
import torchvision
from sklearn .metrics import roc_auc_score,accuracy_score,precision_score,recall_score,f1_score
from PIL import Image
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f78961ea50, raw_cell="import numpy as np
import torch
import torch.nn as.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W5sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a5390290, execution_count=129 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f78961ea50, raw_cell="import numpy as np
import torch
import torch.nn as.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W5sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [130]:
print(torch.cuda.is_available()) # this should return true
torch.cuda.get_device_name() # this should return your graphics card name. Ex) 'NVIDIA RTX A4000'

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a5321b50, raw_cell="print(torch.cuda.is_available()) # this should ret.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W6sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

True


'NVIDIA RTX A4000'

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a53a9b10, execution_count=130 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a5321b50, raw_cell="print(torch.cuda.is_available()) # this should ret.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#W6sZmlsZQ%3D%3D> result='NVIDIA RTX A4000'>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [131]:
# # importing the zipfile module 

"""
from zipfile import ZipFile 

 # loading the temp.zip and creating a zip object 
with ZipFile("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\archive.zip", 'r') as zObject: 
    zObject.extractall() 
    """


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f697415750, raw_cell="# # importing the zipfile module 

"""
from zipfil.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X10sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

'\nfrom zipfile import ZipFile \n\n # loading the temp.zip and creating a zip object \nwith ZipFile("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\archive.zip", \'r\') as zObject: \n    zObject.extractall() \n    '

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a53f36d0, execution_count=131 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f697415750, raw_cell="# # importing the zipfile module 

"""
from zipfil.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X10sZmlsZQ%3D%3D> result='\nfrom zipfile import ZipFile \n\n # loading the temp.zip and creating a zip object \nwith ZipFile("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\archive.zip", \'r\') as zObject: \n    zObject.extractall() \n    '>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [132]:
import pandas as pd
dicom_data = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\dicom_info.csv")

full_mammogram_images = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path
full_mammogram_images.head()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a549d550, raw_cell="import pandas as pd
dicom_data = pd.read_csv("C:\\.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X11sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

1     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.248386...
2     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.267213...
11    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.210396...
12    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.749566...
15    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.987658...
Name: image_path, dtype: object

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a53e3890, execution_count=132 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a549d550, raw_cell="import pandas as pd
dicom_data = pd.read_csv("C:\\.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X11sZmlsZQ%3D%3D> result=1     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.248386...
2     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.267213...
11    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.210396...
12    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.749566...
15    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.987658...
Name: image_path, dtype: object>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [133]:
# load the mass dataset
mass_train = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_train_set.csv')
mass_test = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_test_set.csv')

#mass_train.head()
mass_train.iloc[0][11].split("/")[2]

# fix image paths
def fix_image_path(data):
 #   correct dicom paths to correct image paths
  for index, img in enumerate(data.values):
    img_name = img[11].split("/")[2]
    new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\jpeg\\" + img_name
    data.iloc[index,11] = new_path
      
# apply to datasets
fix_image_path(mass_train)
fix_image_path(mass_test)


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a548cf90, raw_cell="# load the mass dataset
mass_train = pd.read_csv('.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X12sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a541f650, execution_count=133 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a548cf90, raw_cell="# load the mass dataset
mass_train = pd.read_csv('.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X12sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [134]:

calc_train = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_train_set.csv')
calc_test = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_test_set.csv')

fix_image_path(calc_train)
fix_image_path(calc_test)


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a53f3250, raw_cell="
calc_train = pd.read_csv('C:\\Users\\ae423\\OneDr.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X13sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a53f0b90, execution_count=134 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a53f3250, raw_cell="
calc_train = pd.read_csv('C:\\Users\\ae423\\OneDr.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X13sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [135]:
temp_calc_train = calc_train[["image file path","pathology"]]
calc_normal = []
calc_malignant = []
for row,col in temp_calc_train.iterrows():
    if col["pathology"] == "MALIGNANT":
        calc_malignant.append(col["image file path"])
    else:
        calc_normal.append(col["image file path"])

print(len(calc_normal))
print(len(calc_malignant))
print(len(temp_calc_train))


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a1a91210, raw_cell="temp_calc_train = calc_train[["image file path","p.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X14sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

1002
544
1546
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a542a450, execution_count=135 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a1a91210, raw_cell="temp_calc_train = calc_train[["image file path","p.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X14sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [136]:
calc_test

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a1012bd0, raw_cell="calc_test" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X15sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,P_02464,2,RIGHT,MLO,1,calcification,FINE_LINEAR_BRANCHING,CLUSTERED,0,MALIGNANT,4,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_02464_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_02464_RIGHT_MLO_1/1.3.6.1.4.1.9590...
322,P_02498,4,RIGHT,CC,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_02498_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_02498_RIGHT_CC_1/1.3.6.1.4.1.9590....
323,P_02498,4,RIGHT,MLO,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_02498_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_02498_RIGHT_MLO_1/1.3.6.1.4.1.9590...
324,P_02501,3,RIGHT,CC,1,calcification,PLEOMORPHIC,CLUSTERED,0,MALIGNANT,3,C:\Users\ae423\OneDrive - University of Sussex...,Calc-Test_P_02501_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_02501_RIGHT_CC_1/1.3.6.1.4.1.9590....


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a1013550, execution_count=136 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a1012bd0, raw_cell="calc_test" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X15sZmlsZQ%3D%3D> result=    patient_id  breast density left or right breast image view  \
0      P_00038               2                 LEFT         CC   
1      P_00038               2                 LEFT        MLO   
2      P_00038               2                RIGHT         CC   
3      P_00038               2                RIGHT         CC   
4      P_00038               2                RIGHT        MLO   
..         ...          

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [137]:
new_df = mass_train[["image file path","pathology"]]

normal_path = []
malignant_path = []
for row,col in new_df.iterrows():
    if col["pathology"] == "MALIGNANT":
        # put this file to the malignant folder
        malignant_path.append(col["image file path"])
    else:
        normal_path.append(col["image file path"])

print(len(normal_path))
print(len(malignant_path))
print(len(new_df))


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a54be9d0, raw_cell="new_df = mass_train[["image file path","pathology".." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X16sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

681
637
1318
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a54be2d0, execution_count=137 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a54be9d0, raw_cell="new_df = mass_train[["image file path","pathology".." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X16sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [138]:
"""
path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\"
print(len(os.listdir(os.path.join(path,"cbis-test\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-test\\normal"))))
print(len(os.listdir(os.path.join(path,"cbis-train\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-train\\normal"))))
print(len(os.listdir(os.path.join(path,"cbis-val\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-val\\normal"))))
"""

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a0f9bc50, raw_cell=""""
path = "C:\\Users\\ae423\\OneDrive - Universit.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X20sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

'\npath = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\"\nprint(len(os.listdir(os.path.join(path,"cbis-test\\malignant"))))\nprint(len(os.listdir(os.path.join(path,"cbis-test\\normal"))))\nprint(len(os.listdir(os.path.join(path,"cbis-train\\malignant"))))\nprint(len(os.listdir(os.path.join(path,"cbis-train\\normal"))))\nprint(len(os.listdir(os.path.join(path,"cbis-val\\malignant"))))\nprint(len(os.listdir(os.path.join(path,"cbis-val\\normal"))))\n'

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a0f9b650, execution_count=138 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a0f9bc50, raw_cell=""""
path = "C:\\Users\\ae423\\OneDrive - Universit.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X20sZmlsZQ%3D%3D> result='\npath = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\"\nprint(len(os.listdir(os.path.join(path,"cbis-test\\malignant"))))\nprint(len(os.listdir(os.path.join(path,"cbis-test\\normal"))))\nprint(len(os.listdir(os.path.join(path,"cbis-train\\malignant"))))\nprint(len(os.listdir(os.path.join(path,"cbis-train\\normal"))))\nprint(len(os.l

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [139]:
"""
from sklearn.model_selection import train_test_split


temp,test = train_test_split(last_df, test_size=0.1,shuffle= True)
train,val = train_test_split(temp, test_size=0.1,shuffle= True)

print(len(train))
print(len(val))
print(len(test))
"""
last_df = pd.concat([temp_calc_train,new_df])
calc_test = calc_test[["image file path","pathology"]]
mass_test = mass_test[["image file path","pathology"]]
new_test = pd.concat([calc_test,mass_test])

train = last_df[:2160]
val = last_df[2160:]
test_df = new_test


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a2bdcf10, raw_cell=""""
from sklearn.model_selection import train_test.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X21sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a541ea90, execution_count=139 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a2bdcf10, raw_cell=""""
from sklearn.model_selection import train_test.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X21sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [140]:
"""
import shutil

def write_into_train_folder():
    for row,col in train.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\normal")
                # put it into normal folder


def write_into_val_folder():
    for row,col in val.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\normal")
                # put it into normal folder


def write_into_test_folder():
    for row,col in test_df.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\normal")
                # put it into normal folder

write_into_train_folder()
write_into_val_folder()
write_into_test_folder()

"""

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a542db50, raw_cell=""""
import shutil

def write_into_train_folder():
.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X22sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

'\nimport shutil\n\ndef write_into_train_folder():\n    for row,col in train.iterrows():\n        for img_name in os.listdir(col["image file path"]):\n            img_path = col["image file path"] + "\\" + img_name\n            if col["pathology"] == "MALIGNANT":\n                # put this file to the malignant folder\n                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\malignant")\n            else:\n                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\normal")\n                # put it into normal folder\n\n\ndef write_into_val_folder():\n    for row,col in val.iterrows():\n        for img_name in os.listdir(col["image file path"]):\n            img_path = col["image file path"] + "\\" + img_name\n            if col["pathology"] == "MALIGNANT":\n                # put this file to the malignant folder\n                shu

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a542cd10, execution_count=140 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a542db50, raw_cell=""""
import shutil

def write_into_train_folder():
.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X22sZmlsZQ%3D%3D> result='\nimport shutil\n\ndef write_into_train_folder():\n    for row,col in train.iterrows():\n        for img_name in os.listdir(col["image file path"]):\n            img_path = col["image file path"] + "\\" + img_name\n            if col["pathology"] == "MALIGNANT":\n                # put this file to the malignant folder\n                shutil.copy(img_path,"C:\\Users\\ae423\

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [141]:
import wandb
run = wandb.init(
    # Set the project where this run will be logged
    project="Pretrained Output",
    # Track hyperparameters and run metadata
)
wandb.login()


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a531ef10, raw_cell="import wandb
run = wandb.init(
    # Set the proje.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X23sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f789655c90, execution_count=141 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a531ef10, raw_cell="import wandb
run = wandb.init(
    # Set the proje.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X23sZmlsZQ%3D%3D> result=True>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [142]:
class CLAHE():
    
    def __init__(self, clipLimit=2.0, tileGridSize=(16,16)):
        super().__init__
        self.clipLimit = clipLimit
        self.tileGridSize = tileGridSize

    def __call__(self,sample):
      img = np.array(sample)
      img = np.moveaxis(img,0,2)
      imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      imgray = imgray.astype(np.uint16)
      clahe = cv2.createCLAHE(clipLimit=self.clipLimit, tileGridSize=self.tileGridSize)
      new_img = clahe.apply(imgray)
      new_img = cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
      new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
      new_img = new_img.astype(np.float32)
      new_img = np.moveaxis(new_img,2,0)
      tensor = torch.tensor(new_img)
      #print(img.shape)
      return tensor


class GaussianNoise():

    def __init__(self,mean=0,stddev=0.01):
        super().__init__
        self.mean = mean
        self.stddev = stddev

    def __call__(self,sample):
        sample = sample.cpu()
        input_array = sample.data.numpy()

        noise = np.random.normal(loc=self.mean, scale=self.stddev, size=np.shape(input_array))

        out = np.add(input_array, noise)

        output_tensor = torch.from_numpy(out)
        out_tensor = Variable(output_tensor)
        out = out_tensor.cuda()
        out = out.float()
        return out

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a542fe10, raw_cell="class CLAHE():
    
    def __init__(self, clipLim.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X24sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a548dc10, execution_count=142 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a542fe10, raw_cell="class CLAHE():
    
    def __init__(self, clipLim.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X24sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [143]:
import copy

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a1812d10, raw_cell="import copy

class EarlyStopping:
    def __init__.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X25sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a2bee5d0, execution_count=143 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a1812d10, raw_cell="import copy

class EarlyStopping:
    def __init__.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X25sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [144]:

def build_dataset(batch_size):
   
    transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5), # data augmentation
    transforms.RandomRotation(degrees=(0,180)),
    transforms.RandomAffine(degrees = 0, translate = (0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # normalization
    CLAHE(),
    GaussianNoise()

])
    
    train_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train'
    train_dataset = datasets.ImageFolder(train_dir, transforms_train)
    first_128 = list(range(64))
    train_dataloader = torch.utils.data.Subset(train_dataset,first_128)
    train_dataloader = torch.utils.data.DataLoader(train_dataloader, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

    test_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test'
    test_dataset = datasets.ImageFolder(test_dir, transforms_train)
    test_dataloader = torch.utils.data.Subset(test_dataset,first_128)
    test_dataloader = torch.utils.data.DataLoader(test_dataloader, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

    val_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val'
    val_dataset = datasets.ImageFolder(val_dir, transforms_train)
    val_dataloader = torch.utils.data.Subset(val_dataset,first_128)
    val_dataloader = torch.utils.data.DataLoader(val_dataloader, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

    return train_dataloader,test_dataloader,val_dataloader


def build_resnet():
    model_ft = models.resnet50(weights='IMAGENET1K_V1')
    for param in model_ft.parameters():
        param.requires_grad = False
    num_ftrs = model_ft.fc.in_features # 2048
    model_ft.fc = nn.Linear(num_ftrs, 16) # 16 values will be come for the merged model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    return model_ft.to(device)

      

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a1081090, raw_cell="
def build_dataset(batch_size):
   
    transforms.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X26sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a2c37c10, execution_count=144 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a1081090, raw_cell="
def build_dataset(batch_size):
   
    transforms.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X26sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [145]:
numerical_predictions = []
def pathology_encoder(val):
    return "MALIGNANT" if val == "MALIGNANT" else "NORMAL"

calc_df = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_train_set.csv")
c_test = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_test_set.csv")
calc_df["pathology"] = calc_df["pathology"].map(pathology_encoder)
cols_to_be_used = ["breast density","left or right breast","calc type","calc distribution","pathology"]
new_calc_df = calc_df[cols_to_be_used]

mass_df = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_train_set.csv")
m_test = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_test_set.csv")
mass_df["pathology"] = mass_df["pathology"].map(pathology_encoder)
cols_to_be_used = ["breast_density","left or right breast","mass shape","mass margins","pathology"]
new_mass_df = mass_df[cols_to_be_used]
new_mass_df.rename(columns={"breast_density":"breast density"},inplace=True)

bc_df = pd.concat([new_calc_df,new_mass_df])
lbl_encoder = LabelEncoder()
bc_df['left or right breast'] = lbl_encoder.fit_transform(bc_df['left or right breast'])
bc_df['calc type'] = lbl_encoder.fit_transform(bc_df['calc type'])
bc_df["calc distribution"] = lbl_encoder.fit_transform(bc_df['left or right breast'])
bc_df['pathology'] = lbl_encoder.fit_transform(bc_df['pathology'])
bc_df['mass shape'] = lbl_encoder.fit_transform(bc_df['mass shape'])
bc_df['mass margins'] = lbl_encoder.fit_transform(bc_df['mass margins'])

bc_df["breast density"] = bc_df["breast density"].astype(float)
bc_df["breast density"].fillna(np.mean(bc_df["breast density"]),inplace=True)
cols = ["breast density",'left or right breast', 'calc type','calc distribution', 'mass shape', 'mass margins']
scaler = StandardScaler()
bc_df[cols] = scaler.fit_transform(bc_df[cols])

x_train,y_train = bc_df[cols], bc_df["pathology"]
df = pd.concat([c_test,m_test])


cols_to_be_used = ["breast_density","left or right breast","mass shape","mass margins","pathology"]
new_mass_df = mass_df[cols_to_be_used]
new_mass_df.rename(columns={"breast_density":"breast density"},inplace=True)


df['left or right breast'] = lbl_encoder.fit_transform(df['left or right breast'])
df['calc type'] = lbl_encoder.fit_transform(df['calc type'])
df["calc distribution"] = lbl_encoder.fit_transform(df['left or right breast'])
df['pathology'] = lbl_encoder.fit_transform(df['pathology'])
df['mass shape'] = lbl_encoder.fit_transform(df['mass shape'])
df['mass margins'] = lbl_encoder.fit_transform(df['mass margins'])

df["breast density"] = df["breast density"].astype(float)

cols = ["breast density",'left or right breast', 'calc type','calc distribution', 'mass shape', 'mass margins']
scaler = StandardScaler()
df[cols] = scaler.fit_transform(df[cols])
df["breast density"].fillna(np.mean(df["breast density"]),inplace=True)

df = df[["breast density","left or right breast","calc type","calc distribution","pathology","mass shape","mass margins"]] 

x_test,y_test = df[cols], df["pathology"]

class NumericalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(6, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16), # bu output degerini direkt olarak ensembe modele yollayacagiz cunku
            nn.ReLU()
        )

    def forward(self, x):
        output = self.linear_relu_stack(x)
        return output

y_train = np.reshape(y_train,(y_train.shape[0],1))
y_test = np.reshape(y_test,(y_test.shape[0],1))


train_set = np.concatenate((x_train,y_train),axis=1)
test_set = np.concatenate((x_test,y_test),axis=1)
train_set = torch.tensor(train_set, dtype=torch.float32)
test_set = torch.tensor(test_set, dtype=torch.float32)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a2c6b210, raw_cell="numerical_predictions = []
def pathology_encoder(v.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X30sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f6939f1e50, execution_count=145 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a2c6b210, raw_cell="numerical_predictions = []
def pathology_encoder(v.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X30sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [146]:
resnet_predictions = [] # The outputs of predictions will be stored in this array
numerical_predictions = [] # Numerical predictions will be stored in this array

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a2c4c9d0, raw_cell="resnet_predictions = [] # The outputs of predictio.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X31sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a1731e90, execution_count=146 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a2c4c9d0, raw_cell="resnet_predictions = [] # The outputs of predictio.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X31sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [147]:
from sched import scheduler

def numerical_validation(network,val_dataloader,voutputs,vlabels,step=0,epoch_num = 0, mode="validation"):
    model = NumericalClassifier()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    val_loader  = DataLoader(test_set,  batch_size=32, shuffle=False)
    with torch.no_grad():
        for step_num,values in enumerate(val_loader):
            inputs,labels = values[:,:6],values[:,6] 
            inputs,labels = inputs.to(device), labels.to(device)
            if step_num != step:
                continue
            outputs = model(inputs)
            train_ensemble_model(voutputs,outputs,vlabels,epoch_num,"validation")
            validation(network,val_dataloader,step_num+1,epoch_num= epoch_num, mode="validation")



def validation(network,val_dataloader,step=0,epoch_num = 0, mode="validation"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    network.eval()
    network.to(device)
    with torch.no_grad():
        for i, vdata in enumerate(val_dataloader):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(device), vlabels.to(device)
            if i != step:
                continue
            voutputs = torch.squeeze(network(vinputs))
            new_vlabels = torch.tensor(vlabels, dtype=torch.float32).to(device)
            numerical_validation(network,val_dataloader,voutputs,new_vlabels,i,epoch_num,"validation")
     

def train_numerical_classifier(resnet_input,network,loader,optimizer,target,step=0, epoch_num = 0, mode = "training"):
    batch_size = 32
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    optimizer = build_optimizer(NumericalClassifier(),"adam",learning_rate=0.001)
    model = NumericalClassifier()
    model.to(device)
    for batch_num, temp in enumerate(train_loader):
        inputs,labels = temp[:,:6], temp[:,6]
        inputs,labels = inputs.to(device),labels.to(device)
        if batch_num != step:
            continue
        optimizer.zero_grad()      
        numerical_output = model(inputs)
        optimizer.step()
        train_ensemble_model(resnet_input,numerical_output,target, epoch_num, mode)
        train_epoch(network,loader,optimizer,step=batch_num+1)
        # Train ensemble model loss u geriye dondurdukten sonra bir sonraki batch e gecmeden tekrardan resnet train kismina gecilmesi lazim ki orada bir sonraki batche gecilebilsin. 


loss_fn = nn.BCELoss()
def train_epoch(network, loader, optimizer, step = 0, epoch_num = 0, mode = "training"):
    cumu_loss = 0
    network = build_resnet()    
    for batch_number, (data, target) in enumerate(loader): # batch batch butun datayi gezip loss degerlerini hesapliyorum
        # ensembe isini bitirdikten sonra bir sonraki batche gecilmesi gerekiyor bastan degil yani
        if batch_number != step:
            continue
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data,target = data.to(device),target.to(device)
        optimizer.zero_grad()
        network.to(device)
        # ➡ Forward pass
        prediction = network(data)
        prediction = torch.squeeze(prediction)
        new_target = torch.tensor(target, dtype=torch.float32)
        optimizer.step()
        train_numerical_classifier(prediction,network,loader,optimizer,new_target,batch_number, epoch_num, mode) # batch number baslangicta 0 burda



def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        train_dataloader,test_dataloader,val_dataloader = build_dataset(config.batch_size)
        network = build_resnet()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=10)
        es = EarlyStopping()
        best_val_loss = np.inf
        PATH = './cifar_net.pth'

        for epoch in range(config.epochs):
            network.train(True)
            print("Epoch num is", epoch)
            train_epoch(network, train_dataloader, optimizer,step=0,epoch_num=epoch,mode="training")
            print("Validationa gectim epoch sayim",epoch)
            scheduler.step()
            validation(network,val_dataloader,epoch_num=epoch, mode="validation")
        
        
        
        #test()

resnet_predictions = []


class EnsembleModel(nn.Module): # This neural network merges the 

    def __init__(self,resnet_input,numerical_input):
        super(EnsembleModel,self).__init__()
        self.resnet_input = resnet_input
        self.numerical_input = numerical_input
        self.total_params = torch.add(self.resnet_input,self.numerical_input) 
        self.classifier = nn.Linear(16,1)


    def forward(self,param):
        param =  self.total_params
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        param = param.to(device)
        return F.sigmoid(self.classifier(param))   

current_mode = "training"
predictions = []
targets = []
epoch_val = 0   
loss_values = []
val_loss_values = []
def train_ensemble_model(resnet_input,numerical_input,target, epoch_num=0, mode="training"):
    global epoch_val
    global current_mode
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    ensemble_model = EnsembleModel(resnet_input,numerical_input)
    ensemble_model.to(device)
    if mode == "train":
        ensemble_model.train()
    else: # in validation mode
        ensemble_model.eval()
    #print("Parametreler",ensemble_model.total_params.shape)
    i = 0
    optimizer = build_optimizer(NumericalClassifier(),"adam",learning_rate=0.001)
    while epoch_val == epoch_num:
        optimizer.zero_grad()
        prediction_prob = ensemble_model(ensemble_model.total_params) # Buradan sigmoid cikisi 0 ile 1 arasinda bir deger donecek
        prediction_label = prediction_prob.round() # sigmoidden gelen verileri burada 0 ya da 1 e yuvarliyoruz thresholdu 0.5 kabul ederek
        target = torch.reshape(target, (target.shape[0],1))
        
        targets.extend(target.cpu().numpy())
        loss_val = nn.BCELoss()(prediction_label,target)
        # print("Loss val is", loss_val)
        if mode == "training":
            loss_values.append(loss_val.item())
            predictions.extend(prediction_label.detach().cpu().numpy())
            wandb.log({"batch_loss": loss_val.item(), "epoch": epoch_val+1})
            loss_val.backward(retain_graph=True) 
        else: # mode == "validation"
            print("mode validation oldu")
            val_loss_values.append(loss_val.item())
            print(val_loss_values)
            predictions.extend(prediction_label.cpu().numpy())
            wandb.log({"val_batch_loss": loss_val.item(), "epoch": epoch_val+1}) 
        # bu islemden sonra batch bitti ve cikmasi lazim
        break

    if current_mode != mode: # this condition states that if the epoch has finished

        y_true_tensor = torch.tensor(targets)
        y_pred_tensor = torch.tensor(predictions)
        print(y_pred_tensor)
        #print(y_true_tensor)
        #sys.exit()
        if current_mode == "training":
 
            TP = ((y_pred_tensor == 1.) & (y_true_tensor == 1.)).sum().item()
            FP = ((y_pred_tensor == 1.) & (y_true_tensor == 0.)).sum().item()
            FN = ((y_pred_tensor == 0.) & (y_true_tensor == 1.)).sum().item()
            TN = ((y_pred_tensor == 0.) & (y_true_tensor == 0.)).sum().item()

            accuracy = (TP + FN) / (TP + FN + FP + TN)
            precision = TP / (TP + FP) if TP + FP > 0 else 0
            recall = TP / (TP + FN) if TP + FN > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            #auc = np.round(roc_auc_score(target, loss_values), 3)
            avg_loss = sum(loss_values) / len(loss_values)
            wandb.log({"training_loss": avg_loss, "epoch": epoch_val+1})
            wandb.log({"training_accuracy": accuracy, "epoch": epoch_val+1})
            wandb.log({"training_precision": precision, "epoch": epoch_val+1})
            wandb.log({"training_recall": recall, "epoch": epoch_val+1})
            wandb.log({"training_f1_score": f1, "epoch": epoch_val+1})
            loss_values.clear()
            predictions.clear()
            targets.clear()

        if current_mode == "validation":
            #print("Kod buraya girebiliyor")
            TP = ((y_pred_tensor == 1) & (y_true_tensor == 1)).sum().item()
            FP = ((y_pred_tensor == 1) & (y_true_tensor == 0)).sum().item()
            FN = ((y_pred_tensor == 0) & (y_true_tensor == 1)).sum().item()
            TN = ((y_pred_tensor == 0) & (y_true_tensor == 0)).sum().item()

            accuracy = (TP + FN) / (TP + FN + FP + TN)
            precision = TP / (TP + FP) if TP + FP > 0 else 0
            recall = TP / (TP + FN) if TP + FN > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            #auc = np.round(roc_auc_score(target, val_loss_values), 3)
            avg_val_loss = sum(val_loss_values) / len(val_loss_values)
            wandb.log({"validation_loss": avg_val_loss, "epoch": epoch_val+1})
            wandb.log({"validation_accuracy": accuracy, "epoch": epoch_val+1})
            wandb.log({"validation_precision": precision, "epoch": epoch_val+1})
            wandb.log({"validation_recall": recall, "epoch": epoch_val+1})
            wandb.log({"validation_f1_score": f1, "epoch": epoch_val+1})
            val_loss_values.clear()
            predictions.clear()
            targets.clear()
        epoch_val += 1
        current_mode = mode
        
        
        

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f693913610, raw_cell="from sched import scheduler

def numerical_validat.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X32sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a1c72290, execution_count=147 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f693913610, raw_cell="from sched import scheduler

def numerical_validat.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X32sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [148]:
targets

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f69741ca50, raw_cell="targets" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X42sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

[]

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a12a7010, execution_count=148 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f69741ca50, raw_cell="targets" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X42sZmlsZQ%3D%3D> result=[]>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [149]:
# def test():
#     network = build_resnet()
#     best_checkpoint = torch.load('./cifar_net.pth')
#     best_epoch = best_checkpoint["epoch"]
#     print("Best epoch",best_epoch)
    

#     network.eval()
#     y_true = []
#     y_pred = []

#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     train_dataloader,test_dataloader,val_dataloader = build_dataset(32)
#     for epoch in range(best_epoch):
#         with torch.no_grad():
#             for images, labels in test_dataloader:
#                 images = images.to(device)
#                 outputs = network(images)
#                 pred_prob = torch.sigmoid(outputs)
#                 predicted = pred_prob.round()
#                 #_, predicted = torch.max(outputs, 1)
#                 resnet_predictions.extend(outputs.cpu().numpy())
#                 y_pred.extend(predicted.cpu().numpy()) # bu benim ust mlp ye koyacagim input oluyor su an
#                 y_true.extend(labels.cpu().numpy())
            

#             # Convert lists to tensors for calculation
#             y_true_tensor = torch.tensor(y_true)
#             y_pred_tensor = torch.tensor(y_pred)
            
#             # Calculating precision, recall, and F1 score using PyTorch
#             TP = ((y_pred_tensor == 1) & (y_true_tensor == 1)).sum().item()
#             FP = ((y_pred_tensor == 1) & (y_true_tensor == 0)).sum().item()
#             FN = ((y_pred_tensor == 0) & (y_true_tensor == 1)).sum().item()
#             TN = ((y_pred_tensor == 0) & (y_true_tensor == 0)).sum().item()

#             accuracy = (TP + FN) / (TP + FN + FP + TN)
#             precision = TP / (TP + FP) if TP + FP > 0 else 0
#             recall = TP / (TP + FN) if TP + FN > 0 else 0
#             f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
#             auc = np.round(roc_auc_score(y_true, y_pred), 3)

#             print(f'Accuracy: {accuracy}')
#             print(f'Precision: {precision}')
#             print(f'Recall: {recall}')
#             print(f'F1 Score: {f1}')
            
#             wandb.log({"precision":precision})
#             wandb.log({"recall":recall})
#             wandb.log({"f1 score":f1})
#             wandb.log({"AUC":auc})

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7895d4110, raw_cell="# def test():
#     network = build_resnet()
#    .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X33sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f78977b950, execution_count=149 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7895d4110, raw_cell="# def test():
#     network = build_resnet()
#    .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X33sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [150]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam']
        },
    'learning_rate': {
        "values" : [0.001] # 0.0001 vermeyi bi dene bakalim
      },
      'batch_size': {
        "values" : [32]
      },
      "epochs" : {
        "values" : [2]
      }
    }
 
sweep_config['parameters'] = parameters_dict


import pprint
pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config, project="Pretrained Output")

wandb.agent(sweep_id, train)



Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f6973e1050, raw_cell="sweep_config = {
    'method': 'grid'
    }

metri.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X34sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [32]},
                'epochs': {'values': [2]},
                'learning_rate': {'values': [0.001]},
                'optimizer': {'values': ['adam']}}}


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 3wvj63gk
Sweep URL: https://wandb.ai/alphanhilal-university-of-sussex/Pretrained%20Output/sweeps/3wvj63gk


...

Traceback (most recent call last):
  File "C:\Users\ae423\AppData\Local\Temp\ipykernel_16600\3750538245.py", line 98, in train
    validation(network,val_dataloader,epoch_num=epoch, mode="validation")
  File "C:\Users\ae423\AppData\Local\Temp\ipykernel_16600\3750538245.py", line 33, in validation
    numerical_validation(network,val_dataloader,voutputs,new_vlabels,i,epoch_num,"validation")
  File "C:\Users\ae423\AppData\Local\Temp\ipykernel_16600\3750538245.py", line 16, in numerical_validation
    train_ensemble_model(voutputs,outputs,vlabels,epoch_num,"validation")
  File "C:\Users\ae423\AppData\Local\Temp\ipykernel_16600\3750538245.py", line 179, in train_ensemble_model
    accuracy = (TP + FN) / (TP + FN + FP + TN)
               ~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~
ZeroDivisionError: division by zero


batch_loss,██▁
epoch,▁▁▁▁▁▁▁▁██████
training_accuracy,▁
training_f1_score,▁
training_loss,▁
training_precision,▁
training_recall,▁
val_batch_loss,▁
validation_accuracy,▁
validation_f1_score,▁
validation_loss,▁


wandb: ERROR Run vqmnno84 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\ProgramData\anaconda3\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\ae423\AppData\Local\Temp\ipykernel_16600\3750538245.py", line 98, in train
wandb: ERROR     validation(network,val_dataloader,epoch_num=epoch, mode="validation")
wandb: ERROR   File "C:\Users\ae423\AppData\Local\Temp\ipykernel_16600\3750538245.py", line 33, in validation
wandb: ERROR     numerical_validation(network,val_dataloader,voutputs,new_vlabels,i,epoch_num,"validation")
wandb: ERROR   File "C:\Users\ae423\AppData\Local\Temp\ipykernel_16600\3750538245.py", line 16, in numerical_validation
wandb: ERROR     train_ensemble_model(voutputs,outputs,vlabels,epoch_num,"validation")
wandb: ERROR   File "C:\Users\ae423\AppData\Local\Temp\ipykernel_16600\3750538245.py", line 179, in train_ensemble_model
wandb: ERROR     accuracy = (TP +

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f695c6d9d0, execution_count=150 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f6973e1050, raw_cell="sweep_config = {
    'method': 'grid'
    }

metri.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X34sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [151]:
a = [[0],[1],[0],[1],[0]]
a = torch.tensor(a)
# b = [[1],[1],[0],[1],[1]]
# b = torch.tensor(b)

# TP = ((a == 1) & (b == 1)).sum().item()
# FP = ((a == 1) & (b == 0)).sum().item()
# FN = ((a == 0) & (b == 1)).sum().item()
# TN = ((a == 0) & (b == 0)).sum().item()

# accuracy = (TP + FN) / (TP + FN + FP + TN)
# precision = TP / (TP + FP) if TP + FP > 0 else 0
# recall = TP / (TP + FN) if TP + FN > 0 else 0
# f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# print(accuracy)
a

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a14555d0, raw_cell="a = [[0],[1],[0],[1],[0]]
a = torch.tensor(a)
# b .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X43sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

tensor([[0],
        [1],
        [0],
        [1],
        [0]])

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a12a7f50, execution_count=151 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a14555d0, raw_cell="a = [[0],[1],[0],[1],[0]]
a = torch.tensor(a)
# b .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X43sZmlsZQ%3D%3D> result=tensor([[0],
        [1],
        [0],
        [1],
        [0]])>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [152]:
print(len(os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\malignant")))
print(len(os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\normal")))

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7896d70d0, raw_cell="print(len(os.listdir("C:\\Users\\ae423\\OneDrive -.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X35sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

213
209
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f789724710, execution_count=152 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7896d70d0, raw_cell="print(len(os.listdir("C:\\Users\\ae423\\OneDrive -.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X35sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [153]:
def x(step=0):
    for i in range(10):
        if i != step:
            continue
        print(i)
        y(step)


def y(step=0):
    for i in range(10):
        if i != step:
            continue
        print(i)
        z()
        step +=1
        x(step)


def z():
    print("Asama tamam")

x()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a2c24990, raw_cell="def x(step=0):
    for i in range(10):
        if .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X36sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

0
0
Asama tamam
1
1
Asama tamam
2
2
Asama tamam
3
3
Asama tamam
4
4
Asama tamam
5
5
Asama tamam
6
6
Asama tamam
7
7
Asama tamam
8
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
7
Asama tamam
8
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
6
Asama tamam
7
7
Asama tamam
8
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
7
Asama tamam
8
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
5
Asama tamam
6
6
Asama tamam
7
7
Asama tamam
8
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
7
Asama tamam
8
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
6
Asama tamam
7
7
Asama tamam
8
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
7
Asama tamam
8
8
Asama tamam
9
9
Asama tamam
9
Asama tamam
8
Asama tamam
9
9
Asama tamam
9
As

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [154]:
print(len(os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\malignant")))
print(len(os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\normal")))

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1f7a21ce610, raw_cell="print(len(os.listdir("C:\\Users\\ae423\\OneDrive -.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X40sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

213
209
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001F693990DD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1f7a13329d0, execution_count=154 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1f7a21ce610, raw_cell="print(len(os.listdir("C:\\Users\\ae423\\OneDrive -.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X40sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host